In [0]:
%run /Workspace/Users/ndejong@spn.org/Mailchimp_ETL/Production_ETL/01_Bronze_prod


In [0]:
# test_mailchimp_bronze_etl

import unittest
from unittest.mock import patch, MagicMock
import json



class TestMailchimpETL(unittest.TestCase):

    def test_safe_sanitize_list_name(self):
        self.assertEqual(safe_sanitize_list_name("Test List!"), "Test_List_")
        self.assertEqual(safe_sanitize_list_name("🔥Cool_List🔥"), "_Cool_List_")  # fixed expected result

    @patch("__main__.container_client")
    def test_upload_raw_json(self, mock_container_client):
        mock_file_client = MagicMock()
        mock_container_client.get_file_client.return_value = mock_file_client

        folder = "mailchimp_members/test_list/ingestion_date=2024-01-01"
        file = "test.json"
        json_data = '{"hello": "world"}'

        upload_raw_json(folder, file, json_data)

        full_path = f"{folder}/{file}"
        mock_container_client.get_file_client.assert_called_with(full_path)
        mock_file_client.upload_data.assert_called_once_with(json_data.encode("utf-8"), overwrite=True)

    @patch("__main__.client")
    def test_fetch_all_lists(self, mock_client):
        mock_client.lists.get_all_lists.side_effect = [
            {"lists": [{"id": "1", "name": "List1"}], "total_items": 2},
            {"lists": [{"id": "2", "name": "List2"}], "total_items": 2}
        ]

        result = fetch_all_lists()
        self.assertEqual(len(result), 2)
        self.assertEqual(result[0]["name"], "List1")
        self.assertEqual(result[1]["name"], "List2")

    @patch("__main__.upload_raw_json")
    @patch("__main__.client")
    def test_fetch_list_members(self, mock_client, mock_upload_raw_json):
        mock_list = {"id": "123", "name": "Test List"}
        mock_client.lists.get_list_members_info.side_effect = [
            {"members": [{"email": "a@example.com"}]},
            {"members": []}  # Stop paging
        ]

        fetch_list_members(mock_list)

        mock_upload_raw_json.assert_called_once()
        args, _ = mock_upload_raw_json.call_args
        self.assertIn("Test_List", args[0])  # Folder path check
        data_written = json.loads(args[2])
        self.assertEqual(len(data_written), 1)


# Run the tests in this notebook
unittest.main(argv=[''], verbosity=2, exit=False)


In [0]:
class TestMailchimpETLIntegration(unittest.TestCase):

    @patch("__main__.cleanup_old_files")
    @patch("__main__.fetch_and_store_members")
    @patch("__main__.fetch_all_lists")
    def test_run_etl_full_flow(self, mock_fetch_lists, mock_store_members, mock_cleanup):
        mock_fetch_lists.return_value = [
            {"id": "list1", "name": "List One"},
            {"id": "list2", "name": "List Two"}
        ]

        run_mailchimp_bronze_etl()

        mock_fetch_lists.assert_called_once()
        mock_store_members.assert_called_once_with(mock_fetch_lists.return_value)
        mock_cleanup.assert_called_once()

# Run the test
unittest.main(argv=[''], verbosity=2, exit=False)